# Data Cleaning and Wrangling
The audio clips used in this project came from [CREMA-D](https://github.com/CheyneyComputerScience/CREMA-D) and [RAVDESS](https://smartlaboratory.org/ravdess/) which are two databases consiting of lines read by actors portryaing specific emotions. These clips were filtered using voice activity detection then converted into numpy arrays to store within the dataframe `Audio Data.csv`. Overall this process saves a csv with the following features:
- statment: indicates the phase used for the clip
- emotion: intended emotion of audio
- audio: numpy array of audio
- sampling rate: sampling rate of audio
- sex: actor's sex

In [1]:
# Audio Modules
import collections
import contextlib
import sys
import wave
import webrtcvad
import sounddevice as sd
# General Modules
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import zipfile

# Functions
Audio processing in python is a relativley new thing for me so I "found inspiration" (aka tweeked snippets of code) from the projects of others.
- Voice activity detection by [John Wiseman](https://github.com/wiseman)
- Byte to numpy array conversion by [HudsonHuang](https://gist.github.com/HudsonHuang)

In [2]:
def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate

def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)

class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration

def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        #sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                # sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                #sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    #if triggered:
    #    sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    #sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

def VAD(path,ag=1):
    audio, sample_rate = read_wave(path)
    vad = webrtcvad.Vad(int(ag))
    frames = frame_generator(30, audio, sample_rate)
    frames = list(frames)
    segments = vad_collector(sample_rate, 30, 300, vad, frames)
    
    audio_byte = b"".join(segments)
    return (audio_byte,sample_rate)

def byte_to_float(byte):
    # byte -> int16(PCM_16) -> float32
    return pcm2float(np.frombuffer(byte,dtype=np.int16), dtype='float32')

def pcm2float(sig, dtype='float32'):
    """Convert PCM signal to floating point with a range from -1 to 1.
    Use dtype='float32' for single precision.
    Parameters
    ----------
    sig : array_like
        Input array, must have integral type.
    dtype : data type, optional
        Desired (floating point) data type.
    Returns
    -------
    numpy.ndarray
        Normalized floating point data.
    See Also
    --------
    float2pcm, dtype
    """
    sig = np.asarray(sig)
    if sig.dtype.kind not in 'iu':
        raise TypeError("'sig' must be an array of integers")
    dtype = np.dtype(dtype)
    if dtype.kind != 'f':
        raise TypeError("'dtype' must be a floating point type")

    i = np.iinfo(sig.dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig.astype(dtype) - offset) / abs_max

def play(audio_data,n=None):
    '''
    Plays a random audio clip from a set of audio clips. 
    Prints the statement number, emotion, and sex of the actor
    '''
    if n == None:
        n=np.random.randint(0,len(audio_data)+1)
        
    test = audio_data.iloc[n]
    print(test.loc[['statment','emotion','sex']])

    test_audio = byte_to_float(test['audio'])
        
    test_f = test['sampling rate']

    sd.play(test_audio, test_f)
    status = sd.wait()

# Unzip Audio Files

In [3]:
if 'Audio Clips' not in os.listdir():
    with zipfile.ZipFile("Audio Clips.zip","r") as zip_ref:
        zip_ref.extractall()

# Collection and VAD
Use voice activity detection to remove any background noise and put audio into a dataset

In [4]:
data_RAVDESS = pd.DataFrame()
error = []

folder='Audio Clips/RAVDESS/'
count = 0
for actor in os.listdir(folder):
    path = folder+actor+'/'
    for track in os.listdir(path):
        count+=1
        try:
            metadata=[int(x) for x in track[:-4].split('-')]
            audio_byte,rate = VAD(path+track);
            metadata.append(audio_byte)
            metadata.append(rate)
            data_RAVDESS = data_RAVDESS.append([metadata])
        except:
            error.append(track)
        print('%4d of 1440 complete (%d%%)' % (count,(count/1440)*100),end='\r')
            
data_RAVDESS.head()

,0,1,2,3,4,5,6,7,8
0,3,1,1,1,1,1,1,b'\xe6\xff\xfb\xff\xfe\xff5\x00\\\x00:\x00:\x0...,48000
0,3,1,1,1,1,2,1,b'\x1d\x00!\x00\x01\x00\xe8\xff\xff\xff\x15\x0...,48000
0,3,1,1,1,2,1,1,b'E\x00Q\x00S\x00^\x00f\x00b\x00^\x00]\x00S\x0...,48000
0,3,1,1,1,2,2,1,b'\r\xff\x16\xff\x1f\xff\x1e\xff(\xff9\xff@\xf...,48000
0,3,1,2,1,1,1,1,b'\xd3\xff\xeb\xff\xfa\xff\xf9\xff\x19\x00C\x0...,48000


In [5]:
data_CREMA_D = pd.DataFrame()

folder='Audio Clips/CREMA_D/'
count = 0
for track in os.listdir(folder):
    path = folder+track
    count+=1
    try:
        metadata=track[:-4].split('_')
        audio_byte,rate = VAD(path)
        metadata.append(audio_byte)
        metadata.append(rate)
        data_CREMA_D = data_CREMA_D.append([metadata])
    except:
        error.append(track)
    print('%4d of 7442 complete (%d%%)' % (count,(count/7442)*100),end='\r')
        
data_CREMA_D.head()

,0,1,2,3,4,5
0,1001,DFA,ANG,XX,b'\xae\x00\xa9\x00\xb1\x00\xac\x00\xa8\x00\xad...,16000
0,1001,DFA,DIS,XX,b'C\x00\xf3\xff\xdb\xff]\x00\xa8\x00n\x00\x1b\...,16000
0,1001,DFA,FEA,XX,b'\xd2\x00\xa9\x00\xf1\x00\x8d\x00\xe5\x00\xf6...,16000
0,1001,DFA,HAP,XX,b'4\xffP\xff\x0e\xff\xb8\xfe\xb3\xfe\'\xff1\xf...,16000
0,1001,DFA,NEU,XX,b'\xe5\xfe\xf4\xfe\xef\xfe\x06\xff\x19\xff\x14...,16000


# Cleaning
Clean and combine both datasets

In [6]:
# change index to a range of ints
data_RAVDESS.index = pd.RangeIndex(0,stop = len(data_RAVDESS))
data_CREMA_D.index = pd.RangeIndex(0,stop = len(data_CREMA_D))

# drop unnecessary columns
data_RAVDESS = data_RAVDESS.drop(columns=[0,1,3,5])
data_CREMA_D = data_CREMA_D.drop(columns=[3])

# rename columns
data_RAVDESS.columns=['emotion','statment','actorID','audio','sampling rate']
data_CREMA_D.columns=['ActorID','statment','emotion','audio','sampling rate']

# make sure id is an int not an object
data_CREMA_D[['ActorID']] = data_CREMA_D[['ActorID']].astype('int')

data_RAVDESS.info()
data_CREMA_D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435 entries, 0 to 1434
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   emotion        1435 non-null   int64 
 1   statment       1435 non-null   int64 
 2   actorID        1435 non-null   int64 
 3   audio          1435 non-null   object
 4   sampling rate  1435 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 56.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7442 entries, 0 to 7441
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ActorID        7442 non-null   int32 
 1   statment       7442 non-null   object
 2   emotion        7442 non-null   object
 3   audio          7442 non-null   object
 4   sampling rate  7442 non-null   int64 
dtypes: int32(1), int64(1), object(3)
memory usage: 261.8+ KB


## Label Convention
Make sure that both datasets have the same column labels

In [7]:
# Make all the emotions human redable
for i,j in [('ANG','anger'),('DIS','disgust'),('FEA','fear'),('NEU','neutral'),('HAP','happy'),('SAD','sad')]:
    data_CREMA_D['emotion'] = data_CREMA_D['emotion'].str.replace(i,j)
    
for i,j in [(1,'neutral'),(2,'calm'),(3,'happy'),(4,'sad'),(5,'anger'),(6,'fear'),(7,'disgust'),(8,'surprised')]:
    data_RAVDESS['emotion'] = data_RAVDESS['emotion'].replace(i,j)

In [8]:
# Actor gender
CREMA_D = pd.read_csv('CSVs/CREMA_D_Actors.csv',index_col='Unnamed: 0')
data_CREMA_D = data_CREMA_D.merge(CREMA_D[['Sex','ActorID']],on='ActorID',how='inner')
data_CREMA_D = data_CREMA_D.drop(columns='ActorID').rename(columns={'Sex':'sex'})

data_RAVDESS['actorID'] = data_RAVDESS['actorID'] % 2
data_RAVDESS['actorID'] = data_RAVDESS['actorID'].replace(1,'Male').replace(0,'Female')
data_RAVDESS = data_RAVDESS.rename(columns={'actorID':'sex'})

In [9]:
# change the statement convention
for i,j in enumerate(data_CREMA_D['statment'].unique(),start=3):
    data_CREMA_D['statment'] = data_CREMA_D['statment'].replace(j,i)

Overall there are 14 total statements used:

- 1: Kids are talking by the door.
- 2: Dogs are sitting by the door.
- 3: Don't forget a jacket.
- 4: It's eleven o'clock.
- 5: I'm on my way to the meeting.
- 6: I think I have a doctor's appointment.
- 7 I think I've seen this before.
- 8: I would like a new alarm clock.
- 9: I wonder what this is about.
- 10: Maybe tomorrow it will be cold.
- 11: The airplane is almost full.
- 12: That is exactly what happened.
- 13: The surface is slick.
- 14: We'll stop in a couple of minutes.

## Final Check
Make sure the columns and column labels are the same.

In [10]:
data_CREMA_D.head()

,statment,emotion,audio,sampling rate,sex
0,3,anger,b'\xae\x00\xa9\x00\xb1\x00\xac\x00\xa8\x00\xad...,16000,Male
1,3,disgust,b'C\x00\xf3\xff\xdb\xff]\x00\xa8\x00n\x00\x1b\...,16000,Male
2,3,fear,b'\xd2\x00\xa9\x00\xf1\x00\x8d\x00\xe5\x00\xf6...,16000,Male
3,3,happy,b'4\xffP\xff\x0e\xff\xb8\xfe\xb3\xfe\'\xff1\xf...,16000,Male
4,3,neutral,b'\xe5\xfe\xf4\xfe\xef\xfe\x06\xff\x19\xff\x14...,16000,Male


In [11]:
data_RAVDESS.head()

,emotion,statment,sex,audio,sampling rate
0,neutral,1,Male,b'\xe6\xff\xfb\xff\xfe\xff5\x00\\\x00:\x00:\x0...,48000
1,neutral,1,Male,b'\x1d\x00!\x00\x01\x00\xe8\xff\xff\xff\x15\x0...,48000
2,neutral,2,Male,b'E\x00Q\x00S\x00^\x00f\x00b\x00^\x00]\x00S\x0...,48000
3,neutral,2,Male,b'\r\xff\x16\xff\x1f\xff\x1e\xff(\xff9\xff@\xf...,48000
4,calm,1,Male,b'\xd3\xff\xeb\xff\xfa\xff\xf9\xff\x19\x00C\x0...,48000


In [12]:
# append data together
data = data_CREMA_D.append(data_RAVDESS)
data.index = pd.RangeIndex(0,len(data))

In [13]:
count = 1
for i in range(len(data)):
    path = 'Clean Audio/Clip_%04d.wav' % (i)
    audio = data.iloc[i]['audio']
    sample_rate = data.iloc[i]['sampling rate']
    write_wave(path, audio, sample_rate)
    print('%4d of 8877 complete (%d%%)' % (count,(count/8877)*100),end='\r')
    count += 1

In [15]:
# save as a csv
data = data.drop(columns=['audio'])
data.to_csv('CSVs/Audio Data.csv')

# zip clean audio
zipfile.ZipFile('Clean Audio.zip', mode='w').write('Clean Audio')

### Example Clip
uncomment the code below to play a random audio clip from the dataset

In [ ]:
#play(data)